In [5]:
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
import os
import random
from sklearn.ensemble import RandomForestClassifier
import time

from dvclive import Live

## Prepare data

In [7]:
DATA_PATH = 'data/features.csv'

# Create data dir
directory_path = Path('data')
directory_path.mkdir(parents=True, exist_ok=True)

In [8]:
# Number of rows
n_rows = 100
n_cols = 5

# Generate random float data for features
feature_data = np.random.rand(n_rows, n_cols)

# Generate random binary data for the target
target_data = np.random.randint(2, size=n_rows)

# Create a DataFrame for features
df_features = pd.DataFrame(feature_data, columns=[f'feature_{i}' for i in range(1, n_cols + 1)])

# Add the target column to the DataFrame
df_features['target'] = target_data

# Save the DataFrame to a CSV file
df_features.to_csv(DATA_PATH, index=False)

In [9]:
df_features.head(3)

,feature_1,feature_2,feature_3,feature_4,feature_5,target
0,0.828222,0.658647,0.535212,0.429869,0.998945,1
1,0.543274,0.888434,0.889953,0.569668,0.991725,1
2,0.234272,0.050941,0.706619,0.721094,0.499815,0


## Train model

In [10]:
MODEL_PATH = "manifests/manifests.pkl"

directory_path = Path('manifests')
directory_path.mkdir(parents=True, exist_ok=True)

In [11]:
print("Training Random Forest model - START")

# Load the prepared data
data = pd.read_csv(DATA_PATH)

# Extract features and target variable
X = data.drop('target', axis=1)
y = data['target']

# Train the model
model = RandomForestClassifier() # Example model
model.fit(X, y)

# Simulate GridSearch hyperparameter tuning
errors = []
max_leaf_nodes = [5, 50, 500, 5000]
for nodes in max_leaf_nodes:
    error = random.random()
    errors.append(error)
    print(f"Max leaf nodes: {nodes}  \t ➡ Mean Absolute Error:  {error}")

# Create a DataFrame from the lists
datapoints = pd.DataFrame({
    'Max Leaf Nodes': max_leaf_nodes,
    'Error': errors
})

# Save the trained model
joblib.dump(model, MODEL_PATH)

Training Random Forest model - START
Max leaf nodes: 5  	 ➡ Mean Absolute Error:  0.7364304783753433
Max leaf nodes: 50  	 ➡ Mean Absolute Error:  0.6380626314260218
Max leaf nodes: 500  	 ➡ Mean Absolute Error:  0.6742005786004946
Max leaf nodes: 5000  	 ➡ Mean Absolute Error:  0.8744539099027187


['manifests/manifests.pkl']

## Log metrics and Artifacts

In [12]:
LIVE_DIR = 'dvclive' # default

In [14]:

EPOCHS = 20

with Live(dir=LIVE_DIR, 
          report='notebook') as live:

    # Log metrics in multiple epocs 
    for i in range(EPOCHS):
        live.log_metric("mae", i * random.random())
        live.log_metric("segment_A/f1", i * random.random())
        live.next_step()
        time.sleep(1)

    # Log a plot with datapoints 
    live.log_plot(
        "errors_vs_leafs",
        datapoints,
        x="Max Leaf Nodes",
        y="Error",
        template="simple",
        title="Errors vs Max Leaf Nodes")

    # Log Model / Manifest 
    live.log_artifact(
        path=MODEL_PATH,
        type="model",
        name="manifest",
        labels=["bio-1023", "example-rnd-monorepo"],
    )
    
    # Log a Dataset Artifact / Manifest 
    live.log_artifact(
        path=DATA_PATH,
        type="dataset",
        name="data-bio-1023",
        labels=["bio-1023", "example-rnd-monorepo"],
        meta=[{'columns': data.columns.to_list()}]
    )
    
print("Pipeline - bio-1023 - COMPLETE")



<div style="width: 100%;height: 700px;text-align: center">
DVCLive Report
</div>


KeyboardInterrupt: 